# Search and Export ECOSTRESS and EMIT Granules

Based on the type and location of the data you want to collect, you can choose whether to get the "Tiled" or "Swath" land surface temperature and emissivity. 

The data that will be downloaded can be altered in the "Filter by file types" section of each code block. 

Tiled data is best suited for large, regularly gridded datasets where you want to efficently manage and process individual sections of the overall data. Whereas swath data is used to represent data collected along a specific path or track, often with varying resolution and coverage. 

## Search for Tiled Land Surface Temperature and Emissivity

In [11]:
#This code is to search for export various site granules at one time. Granules will be downloaded to a .txt file 
import os
import pandas as pd
import geopandas as gpd
import earthaccess

def process_site(site_name, polygon_path, out_dir, start_date="2025-04-01", end_date="2025-07-17"):
    """
    For a given site:
    - Loads the polygon
    - Searches ECOSTRESS and EMIT granules
    - Filters to relevant file types
    - Saves result URLs in a .txt and .csv file in the site-specific folder
    """
    # Load polygon
    poly = gpd.read_file(polygon_path)
    minx, miny, maxx, maxy = poly.total_bounds

    # Set up temporal range and concept IDs
    date_range = (start_date, end_date)
    concept_ids = [
        "C2408750690-LPCLOUD",  # EMIT L2A Reflectance Tiled
        "C2076090826-LPCLOUD"   # ECOSTRESS L2T LST/Emissivity Tiled
    ]

    # Search for granules in bounding box
    print(f" Date range searched: {start_date} to {end_date}")

    results = earthaccess.search_data(
        concept_id=concept_ids,
        bounding_box=(minx, miny, maxx, maxy),
        temporal=date_range,
        cloud_hosted=True
    )

    # Split ECOSTRESS and EMIT
    emit_granules = [g for g in results if g["umm"]["CollectionReference"]["ShortName"] == "EMITL2ARFL"]
    eco_granules  = [g for g in results if g["umm"]["CollectionReference"]["ShortName"] == "ECO_L2T_LSTE"]

    # Filter by file types
    desired_ecostress_assets = ['_LST.tif', '_QC.tif', '_cloud.tif']
    desired_emit_extensions = ['.nc']

    filtered_urls = []

    for g in eco_granules:
        for url in g.data_links(access="https"):
            if any(url.endswith(tag) for tag in desired_ecostress_assets):
                filtered_urls.append(url)

    for g in emit_granules:
        for url in g.data_links(access="https"):
            if any(url.endswith(ext) for ext in desired_emit_extensions):
                filtered_urls.append(url)

    # Make output folder if needed
    os.makedirs(out_dir, exist_ok=True)

    # Save to .txt
    txt_path = os.path.join(out_dir, f"{site_name}_granules.txt")
    with open(txt_path, "w") as f:
        for url in filtered_urls:
            f.write(url + "\n")

    print(f"🔍 {site_name} search:")
    print(f"   ECOSTRESS granules found: {len(eco_granules)}")
    print(f"   EMIT granules found:     {len(emit_granules)}")
    print(f"   Total granules found:     {len(emit_granules + eco_granules)}")

    # Save to .csv
    df = pd.DataFrame({'url': filtered_urls})
    df.to_csv(os.path.join(out_dir, f"{site_name}_granules.csv"), index=False)

    print(f"✅ {site_name}: Found {len(filtered_urls)} granule files. Saved to {out_dir}")

    return eco_granules, emit_granules



## Search for Swath Land Surface Temperature and Emissivity

In [24]:
#This code is to search for export various site granules at one time. Granules will be downloaded to a .txt file 
import os
import pandas as pd
import geopandas as gpd
import earthaccess

def process_site(site_name, polygon_path, out_dir, start_date="2025-04-01", end_date="2025-06-30"):
    """
    For a given site:
    - Loads the polygon
    - Searches ECOSTRESS and EMIT granules
    - Filters to relevant file types
    - Saves result URLs in a .txt and .csv file in the site-specific folder
    """
    # Load polygon
    poly = gpd.read_file(polygon_path)
    minx, miny, maxx, maxy = poly.total_bounds

    # Set up temporal range and concept IDs 
    date_range = (start_date, end_date)
    concept_ids = [
        "C2408750690-LPCLOUD",   # EMIT L2A Reflectance (swath)
        "C2076114664-LPCLOUD"    # ECOSTRESS L2 Swath LSTE
    ]

    # Search for granules in bounding box
    print(f" Date range searched: {start_date} to {end_date}")

    swath_results = earthaccess.search_data(
        concept_id=concept_ids,
        bounding_box=(minx, miny, maxx, maxy),
        temporal=(start_date, end_date),
        cloud_hosted=True
    )

    print(f"Total SWATH granules found: {len(swath_results)}")


    # Split ECOSTRESS and EMIT
    emit_granules = [g for g in swath_results if g["umm"]["CollectionReference"]["ShortName"].startswith("EMIT")]
    eco_granules  = [g for g in swath_results if g["umm"]["CollectionReference"]["ShortName"].startswith("ECO_L2")]


    # Filter by file types
    desired_swath_ecostress_extensions = ['.h5', '.nc', '.hdf5']
    desired_swath_emit_extensions = ['.nc']

    filtered_urls = []

    for g in eco_granules:
        for url in g.data_links(access="https"):
            if any(url.endswith(tag) for tag in desired_swath_ecostress_extensions):
                filtered_urls.append(url)

    for g in emit_granules:
        for url in g.data_links(access="https"):
            if any(url.endswith(ext) for ext in desired_swath_emit_extensions):
                filtered_urls.append(url)

    # Make output folder if needed
    os.makedirs(out_dir, exist_ok=True)

    # Save to .txt
    txt_path = os.path.join(out_dir, f"{site_name}_granules.csv")
    with open(txt_path, "w") as f:
        for url in filtered_urls:
            f.write(url + "\n")

    print(f"🔍 {site_name} search:")
    print(f"   ECOSTRESS granules found: {len(eco_granules)}")
    print(f"   EMIT granules found:     {len(emit_granules)}")
    print(f"   Total granules found:     {len(emit_granules + eco_granules)}")

    # Save to .csv
    df = pd.DataFrame({'url': filtered_urls})
    df.to_csv(os.path.join(out_dir, f"{site_name}swath_granules.csv"), index=False)

    print(f"✅ {site_name}: Found {len(filtered_urls)} granule files. Saved to {out_dir}")

    return eco_granules, emit_granules



In [13]:
# Update these with your actual file paths
process_site(
    site_name="LCDM",
    polygon_path="/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/LCDMQuadrat.geojson",
    out_dir="/Users/kylamonique/Desktop/JPLFiles/data/LCDM"
)

process_site(
    site_name="SD",
    polygon_path="/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/SDQuadrat.geojson",
    out_dir="/Users/kylamonique/Desktop/JPLFiles/data/SD"
)

process_site(
    site_name="PV",
    polygon_path="/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/PVQuadrat.geojson",
    out_dir="/Users/kylamonique/Desktop/JPLFiles/data/PV"
)


 Date range searched: 2025-04-01 to 2025-07-17
🔍 LCDM search:
   ECOSTRESS granules found: 62
   EMIT granules found:     5
   Total granules found:     67
✅ LCDM: Found 201 granule files. Saved to /Users/kylamonique/Desktop/JPLFiles/data/LCDM
 Date range searched: 2025-04-01 to 2025-07-17
🔍 SD search:
   ECOSTRESS granules found: 57
   EMIT granules found:     1
   Total granules found:     58
✅ SD: Found 174 granule files. Saved to /Users/kylamonique/Desktop/JPLFiles/data/SD
 Date range searched: 2025-04-01 to 2025-07-17
🔍 PV search:
   ECOSTRESS granules found: 59
   EMIT granules found:     1
   Total granules found:     60
✅ PV: Found 180 granule files. Saved to /Users/kylamonique/Desktop/JPLFiles/data/PV


([Collection: {'ShortName': 'ECO_L2T_LSTE', 'Version': '002'}
  Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -119.17378006048253, 'EastBoundingCoordinate': -117.96980924294873, 'NorthBoundingCoordinate': 34.33773312439149, 'SouthBoundingCoordinate': 33.33312764840095}]}}}
  Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2025-04-01T18:54:03.708Z', 'EndingDateTime': '2025-04-01T18:54:55.678Z'}}
  Size(MB): 10.18
  Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_38196_005_11SLT_20250401T185403_0713_01/ECOv002_L2T_LSTE_38196_005_11SLT_20250401T185403_0713_01_water.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002_L2T_LSTE_38196_005_11SLT_20250401T185403_0713_01/ECOv002_L2T_LSTE_38196_005_11SLT_20250401T185403_0713_01_cloud.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ECO_L2T_LSTE.002/ECOv002

In [15]:
import geopandas as gpd
import earthaccess

# Load ROI
poi = gpd.read_file("/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/LCDMQuadrat.geojson")
minx, miny, maxx, maxy = poi.total_bounds

# Date range
date_range = ("2025-04-01", "2025-05-31")

# Swath concept IDs
concept_ids = [
    "C2408750690-LPCLOUD",   # EMIT L2A Reflectance (swath)
    "C2076114664-LPCLOUD"    # ECOSTRESS L2 Swath LSTE :contentReference[oaicite:1]{index=1}
]

# Search swath granules
swath_results = earthaccess.search_data(
    concept_id=concept_ids,
    bounding_box=(minx, miny, maxx, maxy),
    temporal=date_range,
    cloud_hosted=True
)

# Separate swath products
emit_swath = [g for g in swath_results if g["umm"]["CollectionReference"]["ShortName"].startswith("EMIT")]
eco_swath  = [g for g in swath_results if g["umm"]["CollectionReference"]["ShortName"].startswith("ECO_L2_LSTE")]

print(f"Swath granules total: {len(swath_results)}")
print(f"EMIT swath granules: {len(emit_swath)}")
print(f"ECOSTRESS swath granules: {len(eco_swath)}")

# Write swath asset URLs to file
output = "swath_granules.txt"
with open(output, "w") as f:
    for g in swath_results:
        for url in g.data_links(access="https"):
            if url.endswith(('.nc', '.h5', '.hdf5')):
                f.write(url + "\n")

print(f"✅ Swath granule links saved to {output}")


Swath granules total: 41
EMIT swath granules: 5
ECOSTRESS swath granules: 36
✅ Swath granule links saved to swath_granules.txt
